In [5]:
import pandas as pd
df = pd.read_csv('reviews.csv')
df

,target,reviews
0,neg,"plot : two teen couples go to a church party ,..."
1,neg,the happy bastard's quick movie review \ndamn ...
2,neg,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs..."
4,neg,synopsis : a mentally unstable man undergoing ...
...,...,...
1995,pos,wow ! what a movie . \nit's everything a movie...
1996,pos,"richard gere can be a commanding actor , but h..."
1997,pos,"glory--starring matthew broderick , denzel was..."
1998,pos,steven spielberg's second epic film on world w...


In [8]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
d = df.reviews[1]
m = tokenizer.tokenize(d)
m = [' '.join(m)]

In [11]:
rev = [{"id": "1", "language": "en", "text":"the happy bastard s quick movie review damn that y2k bug it s got a head start in this movie starring jamie lee curtis and another baldwin brother william this time in a story regarding a crew of a tugboat that comes across a deserted russian tech ship that has a strangeness to it when they kick the power back on little do they know the power within going for the gore and bringing on a few action sequences here and there virus still feels very empty like a movie going for all flash and no substance we don t know why the crew was really out in the middle of nowhere we don t know the origin of what took over the ship just that a big pink flashy thing hit the mir and of course we don t know why donald sutherland is stumbling around drunkenly throughout here it s just hey let s chase these people around with some robots the acting is below average even from the likes of curtis you re more likely to get a kick out of her work in halloween h20 sutherland is wasted and baldwin well he s acting like a baldwin of course the real star here are stan winston s robot design some schnazzy cgi and the occasional good gore shot like picking into someone s brain so if robots and body parts really turn you on here s your movie otherwise it s pretty much a sunken ship of a movie"}]

In [14]:

import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint='xxxxx',
                                            credentials=CognitiveServicesCredentials('yyyyyy'))

# Analyze the reviews you read from the /data/reviews folder earlier
language_analysis = text_analytics_client.detect_language(documents=rev)

# print detected language details for each review
for review_num in range(len(rev)):
    # print the review id
    print(rev[review_num]['id'])

    # Get the language details for this review
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(' - Language: {}\n - Code: {}\n - Score: {}\n'.format(lang.name, lang.iso6391_name, lang.score))

    # Add the detected language code to the collection of reviews (so we can do further analysis)
    rev[review_num]["language"] = lang.iso6391_name

1
 - Language: English
 - Code: en
 - Score: 1.0



In [15]:
key_phrase_analysis = text_analytics_client.key_phrases(documents=rev)

# print key phrases for each review
for review_num in range(len(rev)):
    # print the review id
    print(rev[review_num]['id'])

    # Get the key phrases in this review
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')

1

Key Phrases:
	 s brain
	 stan winston s robot
	 happy bastard s quick movie review
	 sunken ship
	 robots
	 crew
	 kick
	 acting
	 power
	 baldwin brother william
	 deserted russian tech ship
	 likes of curtis
	 donald sutherland
	 jamie lee curtis
	 occasional good gore
	 halloween h20 sutherland
	 schnazzy cgi
	 y2k bug
	 real star
	 story
	 mir
	 time
	 head
	 big pink flashy thing
	 tugboat
	 work
	 body parts
	 action sequences
	 strangeness
	 people
	 virus
	 substance
	 middle
	 origin




In [16]:
# Use the client and reviews you created previously to get sentiment scores
sentiment_analysis = text_analytics_client.sentiment(documents=rev)

# Print the results for each review
for review_num in range(len(rev)):

    # Get the sentiment score for this review
    sentiment_score = sentiment_analysis.documents[review_num].score

    # classifiy 'positive' if more than 0.5, 
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'

    # print file name and sentiment
    print('{} : {} ({})'.format(rev[review_num]['id'], sentiment, sentiment_score))

1 : negative (0.06427857279777527)


In [17]:
entity_analysis = text_analytics_client.entities(documents=rev)

# Print the results for each review
for review_num in range(len(rev)):
    print(rev[review_num]['id'])
    # Get the named entitites in this review
    entities = entity_analysis.documents[review_num].entities
    for entity in entities:
        # Only print datetime or location entitites
        if entity.type in ['DateTime','Location']:
            link = '(' + entity.wikipedia_url + ')' if entity.wikipedia_id is not None else ''
            print(' - {}: {} {}'.format(entity.type, entity.name, link))

1
 - DateTime: halloween 
